# Day 9
## Part 1


In [1]:
import itertools

def parse_data(s):
    return [int(c) for c in s.strip()]

def uncompress(data):
    filesystem = []
    for file_id, file_size, free_space in zip(
        itertools.count(),
        data[::2],
        # The last file has no free space after it
        data[1::2] + [0]
    ):
        filesystem.extend([file_id] * file_size)
        filesystem.extend([None] * free_space)
    return filesystem

test_data = parse_data("2333133121414131402")
print(uncompress(test_data))

[0, 0, None, None, None, 1, 1, 1, None, None, None, 2, None, None, None, 3, 3, 3, None, 4, 4, None, 5, 5, 5, 5, None, 6, 6, 6, 6, None, 7, 7, 7, None, 8, 8, 8, 8, 9, 9]


In [2]:
def compress(filesystem):
    fs = filesystem.copy()
    i = 0
    j = len(fs) - 1
    while True:
        while fs[i] is not None:
            i += 1
        while fs[j] is None:
            j -= 1
        if i >= j:
            return fs
        fs[i] = fs[j]
        fs[j] = None

print(compress(uncompress(test_data)))

[0, 0, 9, 9, 8, 1, 1, 1, 8, 8, 8, 2, 7, 7, 7, 3, 3, 3, 6, 4, 4, 6, 5, 5, 5, 5, 6, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [3]:
def part_1(data):
    return sum(
        i * x
        for i, x in enumerate(
            y 
            for y in compress(uncompress(data))
            if y is not None)
    )

assert part_1(test_data) == 1928

In [5]:
data = parse_data(open("input").read())
part_1(data)

6367087064415

## Part 2

This is fiddly.|

In [14]:
from dataclasses import dataclass

@dataclass
class Block:
    position: int
    size: int

def blocks(data):
    i = 0
    filesystem = {}
    freespace = []
    for file_id, file_size, free_space in zip(
        itertools.count(),
        data[::2],
        # The last file has no free space after it
        data[1::2] + [0]
    ):
        filesystem[file_id] = Block(i, file_size)
        i += file_size
        freespace.append(Block(i, free_space))
        i += free_space
    return filesystem, freespace

blocks(test_data)

({0: Block(position=0, size=2),
  1: Block(position=5, size=3),
  2: Block(position=11, size=1),
  3: Block(position=15, size=3),
  4: Block(position=19, size=2),
  5: Block(position=22, size=4),
  6: Block(position=27, size=4),
  7: Block(position=32, size=3),
  8: Block(position=36, size=4),
  9: Block(position=40, size=2)},
 [Block(position=2, size=3),
  Block(position=8, size=3),
  Block(position=12, size=3),
  Block(position=18, size=1),
  Block(position=21, size=1),
  Block(position=26, size=1),
  Block(position=31, size=1),
  Block(position=35, size=1),
  Block(position=40, size=0),
  Block(position=42, size=0)])

In [24]:
from collections import deque

def compress_blocks(filesystem, freespace):
    fs = filesystem.copy()
    sp = freespace.copy()
    for f in reversed(fs):
        i = 0
        moved = False
        while not moved and i < len(sp) and sp[i].position < fs[f].position:
            if sp[i].size >= fs[f].size:
                sp[i].size -= fs[f].size
                fs[f].position = sp[i].position
                sp[i].position += fs[f].size
                moved = True
            else:
                i += 1
    return fs    

compress_blocks(*blocks(test_data))

{0: Block(position=0, size=2),
 1: Block(position=5, size=3),
 2: Block(position=4, size=1),
 3: Block(position=15, size=3),
 4: Block(position=12, size=2),
 5: Block(position=22, size=4),
 6: Block(position=27, size=4),
 7: Block(position=8, size=3),
 8: Block(position=36, size=4),
 9: Block(position=2, size=2)}

In [26]:
def checksum_block(file_id, block):
    return sum(
        file_id * (block.position + i)
        for i in range(block.size)
    )

def part_2(data):
    compressed = compress_blocks(*blocks(data))
    return sum(
        checksum_block(x, compressed[x])
        for x in compressed
    )

assert part_2(test_data) == 2858

In [27]:
part_2(data)

6390781891880